<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>


# NPA: Neural News Recommendation with Personalized Attention

NPA \[1\] is a news recommendation model with personalized attention. The core of NPA is a news representation model and a user representation model. In the news representation model we use a CNN network to learn hidden representations of news articles based on their titles. In the user representation model we learn the representations of users based on the representations of their clicked news articles. In addition, a word-level and a news-level personalized attention are used to capture different informativeness for different users.

## Properties of NPA:

- NPA is a content-based news recommendation method.
- It uses a CNN network to learn news representation. And it learns user representations from their clicked news articles.
- A word-level personalized attention is used to help NPA attend to important words for different users.
- A news-level personalized attention is used to help NPA attend to important historical clicked news for different users.

## Data format:

For quicker training and evaluaiton, we sample MINDdemo dataset of 5k users from [MIND small dataset](https://msnews.github.io/). The MINDdemo dataset has the same file format as MINDsmall and MINDlarge. If you want to try experiments on MINDsmall
and MINDlarge, please change the dowload source.
Select the MIND_type parameter from ['large', 'small', 'demo'] to choose dataset.

**MINDdemo_train** is used for training, and **MINDdemo_dev** is used for evaluation. Training data and evaluation data are composed of a news file and a behaviors file. You can find more detailed data description in [MIND repo](https://github.com/msnews/msnews.github.io/blob/master/assets/doc/introduction.md)

### news data

This file contains news information including newsid, category, subcatgory, news title, news abstarct, news url and entities in news title, entities in news abstarct.
One simple example: <br>

`N46466 lifestyle lifestyleroyals The Brands Queen Elizabeth, Prince Charles, and Prince Philip Swear By Shop the notebooks, jackets, and more that the royals can't live without. https://www.msn.com/en-us/lifestyle/lifestyleroyals/the-brands-queen-elizabeth,-prince-charles,-and-prince-philip-swear-by/ss-AAGH0ET?ocid=chopendata [{"Label": "Prince Philip, Duke of Edinburgh", "Type": "P", "WikidataId": "Q80976", "Confidence": 1.0, "OccurrenceOffsets": [48], "SurfaceForms": ["Prince Philip"]}, {"Label": "Charles, Prince of Wales", "Type": "P", "WikidataId": "Q43274", "Confidence": 1.0, "OccurrenceOffsets": [28], "SurfaceForms": ["Prince Charles"]}, {"Label": "Elizabeth II", "Type": "P", "WikidataId": "Q9682", "Confidence": 0.97, "OccurrenceOffsets": [11], "SurfaceForms": ["Queen Elizabeth"]}] []`
<br>

In general, each line in data file represents information of one piece of news: <br>

`[News ID] [Category] [Subcategory] [News Title] [News Abstrct] [News Url] [Entities in News Title] [Entities in News Abstract] ...`

<br>

We generate a word_dict file to tranform words in news title to word indexes, and a embedding matrix is initted from pretrained glove embeddings.

### behaviors data

One simple example: <br>
`1 U82271 11/11/2019 3:28:58 PM N3130 N11621 N12917 N4574 N12140 N9748 N13390-0 N7180-0 N20785-0 N6937-0 N15776-0 N25810-0 N20820-0 N6885-0 N27294-0 N18835-0 N16945-0 N7410-0 N23967-0 N22679-0 N20532-0 N26651-0 N22078-0 N4098-0 N16473-0 N13841-0 N15660-0 N25787-0 N2315-0 N1615-0 N9087-0 N23880-0 N3600-0 N24479-0 N22882-0 N26308-0 N13594-0 N2220-0 N28356-0 N17083-0 N21415-0 N18671-0 N9440-0 N17759-0 N10861-0 N21830-0 N8064-0 N5675-0 N15037-0 N26154-0 N15368-1 N481-0 N3256-0 N20663-0 N23940-0 N7654-0 N10729-0 N7090-0 N23596-0 N15901-0 N16348-0 N13645-0 N8124-0 N20094-0 N27774-0 N23011-0 N14832-0 N15971-0 N27729-0 N2167-0 N11186-0 N18390-0 N21328-0 N10992-0 N20122-0 N1958-0 N2004-0 N26156-0 N17632-0 N26146-0 N17322-0 N18403-0 N17397-0 N18215-0 N14475-0 N9781-0 N17958-0 N3370-0 N1127-0 N15525-0 N12657-0 N10537-0 N18224-0`
<br>

In general, each line in data file represents one instance of an impression. The format is like: <br>

`[Impression ID] [User ID] [Impression Time] [User Click History] [Impression News]`

<br>

User Click History is the user historical clicked news before Impression Time. Impression News is the displayed news in an impression, which format is:<br>

`[News ID 1]-[label1] ... [News ID n]-[labeln]`

<br>
Label represents whether the news is clicked by the user. All information of news in User Click History and Impression News can be found in news data file.


## Global settings and imports


In [1]:
from reco_utils.recommender.deeprec.deeprec_utils import download_deeprec_resources
from reco_utils.recommender.newsrec.newsrec_utils import get_mind_data_set
from reco_utils.recommender.newsrec.io.mind_iterator import MINDIterator
from reco_utils.recommender.newsrec.models.npa import NPAModel
from reco_utils.recommender.newsrec.newsrec_utils import prepare_hparams
import sys
import os
import numpy as np
import zipfile
import pandas as pd
from tqdm import tqdm
import scrapbook as sb
from tempfile import TemporaryDirectory
import tensorflow as tf
tf.get_logger().setLevel('ERROR')  # only show error messages


print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))


System version: 3.6.11 (default, Nov 27 2020, 18:37:51) [MSC v.1916 64 bit (AMD64)]
Tensorflow version: 1.15.4


## Prepare Parameters


In [2]:
epochs = 3
seed = 42
batch_size = 32

# Options: demo, small, large
MIND_type = 'demo'


# Load the dataset


## Download and load data


In [3]:
tmpdir = TemporaryDirectory()
# data_path = tmpdir.name
data_path = os.path.join('dataset', MIND_type)

train_news_file = os.path.join(data_path, 'train', r'news.tsv')
train_behaviors_file = os.path.join(data_path, 'train', r'behaviors.tsv')
valid_news_file = os.path.join(data_path, 'valid', r'news.tsv')
valid_behaviors_file = os.path.join(data_path, 'valid', r'behaviors.tsv')
wordEmb_file = os.path.join(data_path, "utils", "embedding.npy")
userDict_file = os.path.join(data_path, "utils", "uid2index.pkl")
wordDict_file = os.path.join(data_path, "utils", "word_dict.pkl")

yaml_file = os.path.join(data_path, "utils", r'npa.yaml')

mind_url, mind_train_dataset, mind_dev_dataset, mind_utils = get_mind_data_set(MIND_type)

if not os.path.exists(train_news_file):
    download_deeprec_resources(mind_url, os.path.join(
        data_path, 'train'), mind_train_dataset)

if not os.path.exists(valid_news_file):
    download_deeprec_resources(mind_url,
                               os.path.join(data_path, 'valid'), mind_dev_dataset)
if not os.path.exists(yaml_file):
    download_deeprec_resources(r'https://recodatasets.z20.web.core.windows.net/newsrec/',
                               os.path.join(data_path, 'utils'), mind_utils)


### Prepare the BERT embeddings

In [4]:
columnNames = ['news_ID', 'category', 'SubCategory', 'title',
               'abstract', 'URL', 'titleEntities', 'abstractEntities']
df = pd.read_csv(train_news_file , sep='\t',
                 header=None, names=columnNames)

In [5]:
seq_length = 300
num_samples = df.shape[0]

# BERT_tokens = np.zeros((num_samples, seq_length))
XIDs = np.zeros((num_samples, seq_length))
XMask = np.zeros((num_samples, seq_length))

# BERT_tokens.shape



In [6]:
# !pip install transformers
df['title'].count()

26740

In [7]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")
BERT_tokens = np.zeros((num_samples, seq_length))
# dd = tokenizer(df['title'][0], max_length=seq_length, 
#                                     truncation=True,
#                                     return_tensors='pt',
#                                     add_special_tokens=True,
#                                     padding='max_length')

arrShape = np.zeros((num_samples, seq_length))
for i, title in enumerate(df['title']):
    encoded_input = tokenizer(df['title'][i], max_length=seq_length, 
                                    truncation=True,
                                    return_tensors='pt',
                                    add_special_tokens=True,
                                    padding='max_length')
    print(i)
    output = model(**encoded_input)
    
# output = model(**BERT_tokens)
# df['title'][0]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42


In [ ]:
# arrShape.shape

In [ ]:
# # !pip install transformers
# from transformers import AutoTokenizer, AutoModel
# import torch


# #Mean Pooling - Take attention mask into account for correct averaging
# def mean_pooling(model_output, attention_mask):
#     token_embeddings = model_output[0] #First element of model_output contains all token embeddings
#     input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
#     return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# # Sentences we want sentence embeddings for
# sentences = ['This is an example sentence', 'Each sentence is converted']

# # Load model from HuggingFace Hub
# tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
# model = AutoModel.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')

# # Tokenize sentences
# encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# # Compute token embeddings
# with torch.no_grad():
#     model_output = model(**encoded_input)

# # Perform pooling. In this case, max pooling.
# sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# print("Sentence embeddings:")
# print(sentence_embeddings)

In [ ]:
# from transformers import BertTokenizer
# import tensorflow as tf
# from transformers import BertTokenizer, BertModel
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertModel.from_pretrained("bert-base-uncased")


# # tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# for i, title in enumerate(df['title']):
#     tokens = tokenizer.encode_plus(title, 
#                                    max_length=seq_length, 
#                                    truncation=True,
#                                    return_tensors='pt',
#                                    add_special_tokens=True,
#                                    padding='max_length'
#                                    )
#     XIDs[i, :] = tokens['input_ids']
#     XMask[i, :] = tokens['attention_mask']
    

np.save(os.path.join(data_path, "utils", "BERT_embeddings.npy"), BERT_tokens['pooled_output'])

In [ ]:
# BERT_file = os.path.join(data_path, "utils", "BERT_embeddings.npy")
# tt = np.load(BERT_file)
# tt[0]

In [ ]:
wordEmb_file = os.path.join(data_path, "utils", "embedding.npy")
tt = np.load(wordEmb_file, allow_pickle=True)
tt

In [ ]:
# val_n = pd.read_csv(valid_news_file, sep='\t')
# tr_n = pd.read_csv(train_news_file, sep='\t')
# word2vec = np.load(wordEmb_file)

# print(val_n.shape, tr_n.shape, word2vec.shape)

## Create hyper-parameters


In [ ]:
# hparams = prepare_hparams(yaml_file,
#                           wordEmb_file=BERT_file,
#                           wordDict_file=wordDict_file,
#                           userDict_file=userDict_file,
#                           batch_size=batch_size,
#                           epochs=epochs)
# print(hparams)


In [ ]:
iterator = MINDIterator


## Train the NPA model


In [ ]:
model = NPAModel(hparams, iterator, seed=seed)


In [ ]:
print(model.run_eval(valid_news_file, valid_behaviors_file))


In [ ]:
%%time
model.fit(train_news_file, train_behaviors_file,
          valid_news_file, valid_behaviors_file)


In [ ]:
%%time
res_syn = model.run_eval(valid_news_file, valid_behaviors_file)
print(res_syn)


In [ ]:
sb.glue("res_syn", res_syn)


## Save the model


In [ ]:
model_path = os.path.join(data_path, "model")
os.makedirs(model_path, exist_ok=True)

model.model.save_weights(os.path.join(model_path, "npa_ckpt"))


## Output Predcition File

This code segment is used to generate the prediction.zip file, which is in the same format in [MIND Competition Submission Tutorial](https://competitions.codalab.org/competitions/24122#learn_the_details-submission-guidelines).

Please change the `MIND_type` parameter to `large` if you want to submit your prediction to [MIND Competition](https://msnews.github.io/competition.html).


In [ ]:
group_impr_indexes, group_labels, group_preds = model.run_slow_eval(
    valid_news_file, valid_behaviors_file)


In [ ]:
with open(os.path.join(data_path, 'prediction.txt'), 'w') as f:
    for impr_index, preds in tqdm(zip(group_impr_indexes, group_preds)):
        impr_index += 1
        pred_rank = (np.argsort(np.argsort(preds)[::-1]) + 1).tolist()
        pred_rank = '[' + ','.join([str(i) for i in pred_rank]) + ']'
        f.write(' '.join([str(impr_index), pred_rank]) + '\n')


In [ ]:
f = zipfile.ZipFile(os.path.join(
    data_path, 'prediction.zip'), 'w', zipfile.ZIP_DEFLATED)
f.write(os.path.join(data_path, 'prediction.txt'), arcname='prediction.txt')
f.close()


In [ ]:
data_path

## Reference

\[1\] Chuhan Wu, Fangzhao Wu, Mingxiao An, Jianqiang Huang, Yongfeng Huang and Xing Xie: NPA: Neural News Recommendation with Personalized Attention, KDD 2019, ADS track.<br>
\[2\] Wu, Fangzhao, et al. "MIND: A Large-scale Dataset for News Recommendation" Proceedings of the 58th Annual Meeting of the Association for Computational Linguistics. https://msnews.github.io/competition.html <br>
\[3\] GloVe: Global Vectors for Word Representation. https://nlp.stanford.edu/projects/glove/
